# Assignment 2

In this assignment you will implement ResNet18.
Read the comments carefully and insert your code where you see: <br><br><b>##### START OF YOUR CODE #####</b><br><br><b>##### END OF YOUR CODE #####</b><br><br>or for the inline codes you will see<br><br><b>##### INSERT YOUR CODE HERE #####</b>

### The architecture of ResNet-18 is shown in the table.
First, we will define a convolutional block with skip connection. Then, create the model using these blocks.<br><br>
<img src="https://www.researchgate.net/profile/Paolo-Napoletano/publication/322476121/figure/tbl1/AS:668726449946625@1536448218498/ResNet-18-Architecture.png" width="500" alt="ResNet18 Architecture">

<br><sup>Image ref: Napoletano, Paolo, et al. ‘Anomaly Detection in Nanofibrous Materials by CNN-Based Self-Similarity’. Sensors (Basel, Switzerland), vol. 18, 01 2018, https://doi.org10.3390/s18010209.</sup>

#### I. ConvBlock
<img src="https://www.researchgate.net/publication/334301817/figure/fig3/AS:778452965801986@1562609058538/Residual-block-of-ResNet18-with-a-1-1-convolutional-mapping-based-residual-unit-and.png"><br>
ResNet consists of convolutional (a) and identity (b) blocks. For ResNet-18 we will only use convolutional blocks. In this step you will write a class for convolutional block. The arguments will be:

* ch_in: input channels
* ch_out: output channels
* s: strides
* act: activation function

The options for activation function are "relu", "leaky_relu" and "gelu".
<br><br>
<sup>Image ref: Owais, Muhammad, et al. ‘Artificial Intelligence-Based Classification of Multiple Gastrointestinal Diseases Using Endoscopy Videos for Clinical Diagnosis’. Journal of Clinical Medicine, vol. 8, 07 2019, p. 986, https://doi.org10.3390/jcm8070986.</sup>

My learnings:
The choice of the number of input and output channels is a hyperparameter also.
The input channels: RGB
The output channels: the number of filters that is applied
Stride: the step size, also affects the output size
Smaller strides preserve more information but require more computation. Larger strides reduce the spatial dimensions, but they are computationally more efficient.

In [11]:
import torch
from torch import nn
from torch.nn import functional as F

class ConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out, s, act):
      super(ConvBlock,self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####

      #((input size + 2padding - kernel size)/stride)+1 = output size
      #Given the calculation above, the paddings for the different layers has been calculated.

      #Convolutional Layer 1
      self.conv_layer1 = nn.Conv2d(ch_in, ch_out, 3, stride = s, padding=1)
      #Batch Normalization 1
      self.bn1 = nn.BatchNorm2d(ch_out)
      #Activation Function
      self.act = act
      #Convolutional Layer 2
      self.conv_layer2 = nn.Conv2d(ch_out, ch_out, 3, stride = s, padding=1)
      #Batch Normalization 2
      self.bn2 = nn.BatchNorm2d(ch_out)

      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####
      #First Convolutional Layer
      X = self.conv_layer1(X)
      X = self.bn1(X)
      X = self.act(X)
      #Second Convolutional Layer
      X = self.conv_layer2(X)
      X = self.bn2(X)
      X = self.act(X)
      ##### END OF YOUR CODE #####
      return X

#### II. ResNet18 class
Use the ConvBlock class to create ResNet18.
* Add batch normalization and activation function after all convolutional layers.
* Examine the output sizes in the table and use paddings and strides where needed. (Note that the input size is 224 x 224 in this example)
* Add a drop-out layer after average pooling.
* Fully connected layer should be 512 x 1 as we have only 2 classes and we will use sigmoid function as the final activation layer.

<img src="https://www.researchgate.net/profile/Paolo-Napoletano/publication/322476121/figure/tbl1/AS:668726449946625@1536448218498/ResNet-18-Architecture.png" width="500" alt="ResNet18 Architecture">

Padding Calculation:

((input size + 2padding - kernel size)/stride)+1 = output size

In [12]:
class ResNet18(nn.Module):
    def __init__(self, act, drop_rate):
      super(ResNet18, self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####
      #((input size + 2padding - kernel size)/stride)+1 = output size
      #Given the calculation above, the paddings for the different layers has been calculated.

      #First Layer
      #It is different from the ConvBlock.
      self.conv1 = nn.Conv2d(1, 64, 7, stride = 2, padding=3)
      self.bn = nn.BatchNorm2d(64)
      self.act = act

      #Three activation functions:
      if act == "relu":
            self.act = nn.ReLU()
      elif act == "leaky_relu":
            self.act = nn.LeakyReLU()
      else:
            self.act = nn.GELU()

      #Maxpool layer
      self.max_pool = nn.MaxPool2d(3, stride = 2, padding = 1)
      #Second Layer
      self.conv2_x = ConvBlock(64, 64, 2, self.act)
      #Third Layer
      self.conv3_x = ConvBlock(64, 128, 2, self.act)
      #Fourth Layer
      self.conv4_x = ConvBlock(128, 256, 2, self.act)
      #Fifth Layer
      self.conv5_x = ConvBlock(256, 512, 2, self.act)

      #Average Pooling Layer
      self.average_pool = nn.AvgPool2d(7)
      #Dropuout Layer
      self.dropout = nn.Dropout(drop_rate)
      #Fully Connected Layer
      self.fully_connected = nn.Linear(512,1000)
      #Activation Layer - Softmax
      self.activation = nn.Softmax(1000)

      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####
      X = self.conv1(X)
      X = self.bn(X)
      X = self.act(X)

      X = self.conv2_x(X)
      X = self.conv3_x(X)
      X = self.conv4_x(X)
      X = self.conv5_x(X)

      X = self.average_pool(X)
      X = self.dropout(X)
      X = self.fully_connected(X)
      X = self.activation(X)
      ##### END OF YOUR CODE #####
      return X

In [13]:
# Print the model
model = ResNet18("relu", .5)
print(model)

ResNet18(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2_x): ConvBlock(
    (conv_layer1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (conv_layer2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3_x): ConvBlock(
    (conv_layer1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (conv_layer2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn2): Batch

In [ ]:
from google.colab import files
f = files.upload()

# Convert ipynb to html
import subprocess
file0 = list(f.keys())[0]
_ = subprocess.run(["pip", "install", "nbconvert"])
_ = subprocess.run(["jupyter", "nbconvert", file0, "--to", "html"])

# download the html
files.download(file0[:-5]+"html")